In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader, random_split

In [2]:
from torchvision.models import resnet50, ResNet50_Weights
resnet50 = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 68.2MB/s]


In [3]:
for param in resnet50.parameters():
    param.requires_grad = False

In [4]:
n = resnet50.fc.in_features
resnet50.fc = nn.Linear(n,4)

In [5]:
transform = models.ResNet50_Weights.DEFAULT.transforms()

In [6]:
path = '/kaggle/input/corn-or-maize-leaf-disease-dataset/data'
dataset = datasets.ImageFolder(root=path, transform=transform)

In [7]:
train_len = int(0.8*len(dataset))
val_len = int(0.1*len(dataset))
test_len = len(dataset)-train_len-val_len

In [8]:
train_data, val_data, test_data = random_split(dataset, [train_len,val_len,test_len])

In [9]:
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False) 

In [32]:
for i, (image,label) in enumerate(train_data):
    break
image.shape

torch.Size([3, 224, 224])

In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
resnet50 = resnet50.to(device)

In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(resnet50.fc.parameters(), lr=0.001, weight_decay=0.0001)

In [29]:
resnet50

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [25]:
def train_model(model, criterion, optimizer, epochs):

    for epoch in range(1,epochs+1):
        model.train()
        running_loss = 0.0
        train_corr=0.0
        val_corr=0.0
        for images, labels in train_loader:
            images = images.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            t,y_pred = torch.max(outputs.data,1)
            train_corr+=(y_pred==labels).sum().item()
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        train_accuracy = train_corr/len(train_loader.dataset)
        
        print(f'Epoch {epoch}/{epochs}, Loss: {epoch_loss:.4f}, Train accuracy: {train_accuracy}')

        model.eval()
        val_loss = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images = images.to(device)
                labels = labels.to(device)
                outputs = model(images)
                t,y_val = torch.max(outputs.data,1)
                val_corr+=(y_val==labels).sum().item()
                loss = criterion(outputs, labels)
                val_loss += loss.item() * images.size(0)

        val_loss = val_loss / len(val_loader.dataset)
        val_acc = val_corr/len(val_loader.dataset)
        
        print(f'Validation Loss: {val_loss:.4f}, Val Accuracy:{val_acc}')

    return model

In [26]:
epochs = 25
trained_model = train_model(resnet50, criterion, optimizer, epochs)

Epoch 1/25, Loss: 0.2465, Train accuracy: 0.9202985074626866
Validation Loss: 0.2147, Val Accuracy:0.9258373205741627
Epoch 2/25, Loss: 0.2147, Train accuracy: 0.9313432835820895
Validation Loss: 0.1902, Val Accuracy:0.930622009569378
Epoch 3/25, Loss: 0.1940, Train accuracy: 0.9402985074626866
Validation Loss: 0.1769, Val Accuracy:0.9282296650717703
Epoch 4/25, Loss: 0.1782, Train accuracy: 0.9435820895522388
Validation Loss: 0.1628, Val Accuracy:0.937799043062201
Epoch 5/25, Loss: 0.1588, Train accuracy: 0.9480597014925373
Validation Loss: 0.1573, Val Accuracy:0.937799043062201
Epoch 6/25, Loss: 0.1514, Train accuracy: 0.9522388059701492
Validation Loss: 0.1521, Val Accuracy:0.9401913875598086
Epoch 7/25, Loss: 0.1433, Train accuracy: 0.9546268656716418
Validation Loss: 0.1476, Val Accuracy:0.9449760765550239
Epoch 8/25, Loss: 0.1370, Train accuracy: 0.9549253731343283
Validation Loss: 0.1395, Val Accuracy:0.9354066985645934
Epoch 9/25, Loss: 0.1302, Train accuracy: 0.957014925373134

In [27]:
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Test Accuracy: {accuracy:.2f}%')  

In [28]:
evaluate_model(trained_model, test_loader)

Test Accuracy: 93.57%
